# Near matching tests
Near matching as of 2016-07-23 (djb)

## Analytic framework
The witnesses start and end with perfect matches (*abcd* and *efgh*, respectively). Witness **A** has one token in the middle (*0123*) and witness **B** has two (*012x*, *01xx*) or three (*012x*, *01xx*, *0xxx*). The two or three candidates for alignment in witness **B** are all partial matches to the middle token in **A**, with different degrees of similarity. All permutations of the candidates in **B** are tested to determine whether **A** is aligned with the correct one.

## Summary of results

1. When there are just two options (left or right), the result is always correct.
1. When there are three options:
    1. The result is correct if the best match is on the far left, since the token never moves.
    1. The result is correct if the best match is on the far right. The token moves to the far right on the first comparison and is never moved again because it's out of scope.
    1. If the best match is in the middle, the result depends on the relative strength of the left and right matches.
        1. If the right match is stronger than the left, the token will move there on the first comparison and then be out of scope, and not available to be moved back to the middle.
        1. If the left match is stronger than the right, the token will remain in its original (far left) place on the first comparison, and the problem then reduces to two options, which always produces the correct result.
1. Situations with more than three options are not tested, but should behave similarly to situations with three options. Specifically, the alignment will fail whenever a token is moved to the right *over* an even stronger match, since the rightward movement takes it out of scope for future movement.
1. All testing was done with exac

General conclusion: Because we always compare far left to far right, whenever there's a stronger match in the middle than on the far right but the far right is stronger than the far left, we will incorrectly move the token over the best match in the middle stronger match to the right, and it will subsequently be out of scope, and will therefore remain in the wrong final location.

## Without near matching, candidate always stays left, even if right is closer
Here *0123* in **A** is closer to *012x* (right) than to *01xx* (left), but it stays left anyway.

In [47]:
%reload_ext autoreload
%autoreload 2
from collatex import *
collation = Collation()
collation.add_plain_witness("A", "abcd 0123 efgh")
collation.add_plain_witness("B", "abcd 01xx 012x efgh")
alignment_table = collate(collation, segmentation=False)
print(alignment_table)

+---+------+------+------+------+
| A | abcd | 0123 | -    | efgh |
| B | abcd | 01xx | 012x | efgh |
+---+------+------+------+------+


## With near matching and two choices, candidate is aligned correctly

In the example below, *0123* in **A** is closer to *012x* (left) in **B**, and it correctly stays left.

In [48]:
# Two candidates
# With near matching, it goes to the closer match, whether that's left or right
# Closer match is left, no movement
collation = Collation()
collation.add_plain_witness("A", "abcd 0123 efgh")
collation.add_plain_witness("B", "abcd 012x 01xx efgh")
alignment_table = collate(collation, near_match=True, segmentation=False)
print(alignment_table)

+---+------+------+------+------+
| A | abcd | 0123 | -    | efgh |
| B | abcd | 012x | 01xx | efgh |
+---+------+------+------+------+


In the example below, *0123* in **A** is closer to *012x* (right) in **B**, and it correctly moves right.

In [49]:
# Two candidates
# With near matching, it goes to the closer match, whether that's left or right
# Same input as above, but closer match is right, so moves
collation = Collation()
collation.add_plain_witness("A", "abcd 0123 efgh")
collation.add_plain_witness("B", "abcd 01xx 012x efgh")
alignment_table = collate(collation, near_match=True, segmentation=False)
print(alignment_table)

+---+------+------+------+------+
| A | abcd | -    | 0123 | efgh |
| B | abcd | 01xx | 012x | efgh |
+---+------+------+------+------+


## With near matching and three or more choices, one of the six options produces incorrect output

### If the closest match is left, the candidate correctly always stays left

In [50]:
# Three candidates, closest is left, match rank 0 1 2 (0 is closest)
# Should stay left; succeeds
collation = Collation()
collation.add_plain_witness("A", "abcd 0123 efgh")
collation.add_plain_witness("B", "abcd 012x 01xx 0xxx efgh")
alignment_table = collate(collation, near_match=True, segmentation=False)
print(alignment_table)

+---+------+------+------+------+------+
| A | abcd | 0123 | -    | -    | efgh |
| B | abcd | 012x | 01xx | 0xxx | efgh |
+---+------+------+------+------+------+


In [51]:
# Three candidates, closest is left, match rank 0 2 1 (0 is closest)
# Should stay left; succeeds
collation = Collation()
collation.add_plain_witness("A", "abcd 0123 efgh")
collation.add_plain_witness("B", "abcd 012x 0xxx 01xx efgh")
alignment_table = collate(collation, near_match=True, segmentation=False)
print(alignment_table)

+---+------+------+------+------+------+
| A | abcd | 0123 | -    | -    | efgh |
| B | abcd | 012x | 0xxx | 01xx | efgh |
+---+------+------+------+------+------+


### If the closest match is right, the candidate correctly always moves right
Stepwise procedure is that:

1. Candidate (*0123*) starts on the left because there is no perfect match
1. Token at **B** aligned with gap in **A** on far right (*012x*) is compared to token in **B** (*01xx*) aligned with candidate on far left; candidate moves far right because *0123* is closer to *012x* (new position) than to *01xx* (old position)
1. Pointer advances to middle (token *0xxx* in **B**); the token in **A** that was moved (*0123*) is now out of scope
1. Gap in **A** in middle (opposite *0xxx* in **B**) is compared to new far left in **A** (*abcd*), but new far left in **A** has a perfect match at its rank, so it doesn't move
1. Pointer advances to left (token *01xx* in **B**); the token in **B** at (*0xxx*), which we just checked, is now out of scope
1. Gap in **A** on left (opposite *01xx* in **B**) is compared to new far left (*abcd*, which still has a perfect match at its rank, so it still doesn't move
1. No more gaps

In [52]:
# Three candidates, closest is right, match rank 1 2 0 (0 is closest)
# Should go right; succeeds
collation = Collation()
collation.add_plain_witness("A", "abcd 0123 efgh")
collation.add_plain_witness("B", "abcd 01xx 0xxx 012x efgh")
alignment_table = collate(collation, near_match=True, segmentation=False)
print(alignment_table)

+---+------+------+------+------+------+
| A | abcd | -    | -    | 0123 | efgh |
| B | abcd | 01xx | 0xxx | 012x | efgh |
+---+------+------+------+------+------+


The stepwise procedure is the same below. The first comparison (far left to far right) finds a closer match in the far right, so the token moves right and is then out of scope when the pointer advances. The remaining gaps in **A** are all evaluated to determine whether to move *abcd*, but because *abcd* has an exact match at its rank, it doesn't move.

In [53]:
# Three candidates, closest is right, match rank 2 1 0 (0 is closest)
# Should go right; succeeds
collation = Collation()
collation.add_plain_witness("A", "abcd 0123 efgh")
collation.add_plain_witness("B", "abcd 0xxx 01xx 012x efgh")
alignment_table = collate(collation, near_match=True, segmentation=False)
print(alignment_table)

+---+------+------+------+------+------+
| A | abcd | -    | -    | 0123 | efgh |
| B | abcd | 0xxx | 01xx | 012x | efgh |
+---+------+------+------+------+------+


## If the closest match is in the middle, the result may be correct or incorrect

### If the closest match is in the middle and the worst match is on the far right, the test succeeds
The steps are:

1. Compare the far left reading in **A** (*0123*) to the **B** readings at its current (far left) position (*01xx*) and opposite the gap on the far right (*0xxx*). Since the current position is a closer match, the token stays put.
1. The pointer advances and the gap on the far right is now out of scope.
1. The problem is now reduced to two choices, which always succeeds, as shown above. The reading on the far left in **A** (*0123*) is compared to the reading at **B** in its current position (*01xx*) and the reading at the gap in **B** in the middle (*012x*), where the pointer is. 
1. The reading in **B** at the gap position is closer, so the token moves right.
1. The pointer advances and the token we just moved is now out of scope. 
1. We compare the reading now at the left in **A** (*abcd*) to the **B** reading in its current position and at the gap where the pointer is (*01xx*). Since it has an exact match in its current position, it stays where it is.
1. No more gaps.

In [54]:
# Three candidates, closest is middle, match rank 1 0 2 (0 is closest)
# Should go right; succeeds
collation = Collation()
collation.add_plain_witness("A", "abcd 0123 efgh")
collation.add_plain_witness("B", "abcd 01xx 012x 0xxx efgh")
alignment_table = collate(collation, near_match=True, segmentation=False)
print(alignment_table)

+---+------+------+------+------+------+
| A | abcd | -    | 0123 | -    | efgh |
| B | abcd | 01xx | 012x | 0xxx | efgh |
+---+------+------+------+------+------+


# If the closest match is in the middle and the worst match is on the far left, the test FAILS
The steps are:

1. Compare the far left reading in **A** (*0123*) to the **B** readings at its current (far left) position (*012x*) and opposite the gap on the far right (*0x12x*). Since the far right position is a closer match, the token moves right.
1. The pointer advances and the token we just moved to the far right is now out of scope.
1. The **A** token on the far left is now *abcd*. Since it has an exact match at its current position, it doesn't move when we compare it first to the **B** reading of *0xxx* and then to the **B** reading of *01xx*.
1. No more gaps.

In [55]:
# Three candidates, closest is middle, match rank 2 0 1 (0 is closest)
# Should go right; succeeds
collation = Collation()
collation.add_plain_witness("A", "abcd 0123 efgh")
collation.add_plain_witness("B", "abcd 01xx 0xxx 012x efgh")
alignment_table = collate(collation, near_match=True, segmentation=False)
print(alignment_table)

+---+------+------+------+------+------+
| A | abcd | -    | -    | 0123 | efgh |
| B | abcd | 01xx | 0xxx | 012x | efgh |
+---+------+------+------+------+------+
